<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/mlflow/summarization/T5_large_Evaluation_multi_news_summarization_mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!pip install mlflow pyngrok  --quiet
! pip install  evaluate  textstat tiktoken -q
! pip install psutil pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00


In [3]:
# Transformers installation
! pip install -q --disable-pip-version-check py7zr sentencepiece loralib peft trl
! pip install -q    wandb bitsandbytes
! pip install datasets evaluate rouge_score -q
! pip install transformers[torch] -q
! pip install accelerate -U -q
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [4]:
from google.colab import userdata
NGROK_AUTH_TOKEN  = userdata.get('NGROK')



In [5]:

import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers

import torch
import torch.nn as nn
from google.colab import userdata

In [6]:

from google.colab import output
output.enable_custom_widget_manager()

from transformers.utils import logging


In [7]:
logging.set_verbosity_warning()

os.environ["TRANSFORMERS_VERBOSITY"] = "warning"

In [8]:


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device


'cuda:0'

# Load multi_news dataset
https://huggingface.co/datasets/multi_news

In [9]:
from datasets import load_dataset

dataset  = load_dataset("multi_news", trust_remote_code=True)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [11]:

print(f"Train dataset size: {len(dataset['train'])}")
print(f"test dataset size: {len(dataset['test'])}")
print(f"Validation dataset size: {len(dataset['validation'])}")

Train dataset size: 44972
test dataset size: 5622
Validation dataset size: 5622


In [12]:
dataset['train'][100]['document']

'Katy Perry is all about breaking conventional beauty rules, from her love of everything technicolor and coated in glitter, to her no-brows, black lipstick Met Gala look. So, of course, the pop star — and face of CoverGirl — was the perfect person to help announce that the beauty brand has named its first-ever male CoverGirl, social media star James Charles. \n \n According to a press release from the brand, all CoverGirls “are role models and boundary-breakers, fearlessly expressing themselves, standing up for what they believe, and redefining what it means to be beautiful,” and who better to embody that ethos than Instagram sensation James Charles. After launching his beauty account a year ago, the teen has since quickly attracted hundreds of thousands of followers (427,000 to be exact) thanks to his unique, transformative approach to makeup artistry. \n \n RELATED PHOTOS: Katy Perry’s Most Outrageous Twitpics \n \n While Charles’ partnership with the brand kicks off today, we’ll hav

In [13]:

len(dataset['train'][100]['document'])

6217

In [14]:

len(dataset['train'][100]['summary'])

1268

In [15]:
import transformers
from mlflow.models import infer_signature
from mlflow.transformers import generate_signature_output
import locale
import mlflow
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [16]:
model_uri = "runs:/a3cb2376e04b48c8b654ac42401dd387/text_summarizer"

In [17]:
MLFLOW_TRACKING_URI="databricks"
# Specify the workspace hostname and token
DATABRICKS_HOST="https://adb-2467347032368999.19.azuredatabricks.net/"
DATABRICKS_TOKEN=userdata.get('DATABRCKS_TTOKEN')

In [18]:


if "MLFLOW_TRACKING_URI" not in os.environ:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI
if "DATABRICKS_HOST" not in os.environ:
    os.environ["DATABRICKS_HOST"] = DATABRICKS_HOST
if "DATABRICKS_TOKEN" not in os.environ:
    os.environ["DATABRICKS_TOKEN"] = DATABRICKS_TOKEN

In [19]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [20]:

mlflow.set_experiment("/Users/***REMOVED***/summarization_fine_tuning")


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/837187481682972', creation_time=1732208495910, experiment_id='837187481682972', last_update_time=1732210306651, lifecycle_stage='active', name='/Users/***REMOVED***/summarization_fine_tuning', tags={'mlflow.experiment.sourceName': '/Users/***REMOVED***/summarization_fine_tuning',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': '***REMOVED***',
 'mlflow.ownerId': '1331640755799986'}>

In [21]:
mlflow.end_run()

In [22]:
summarization_components = mlflow.transformers.load_model(
    model_uri, return_type="components"
)

2024/11/21 17:38:09 INFO mlflow.transformers: 'runs:/a3cb2376e04b48c8b654ac42401dd387/text_summarizer' resolved as 'dbfs:/databricks/mlflow-tracking/837187481682972/a3cb2376e04b48c8b654ac42401dd387/artifacts/text_summarizer'


2024/11/21 17:38:10 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
summarization_components.keys()

dict_keys(['task', 'framework', 'device', 'torch_dtype', 'model', 'tokenizer'])

In [24]:
import torch
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [25]:
reconstructed_pipeline = transformers.pipeline(**summarization_components)

In [26]:
test1= dataset['test'][100]['document']

In [27]:
reconstructed_pipeline(test1)

[{'summary_text': '0:58 KC\'s worst floods 1:54 KC floods: a history of the city Pause 1:55 KC Floods: A history of Kansas City 1:56 KC Firefighters recall worst flood in history 1:57 KC firefighters remember worst flood 1:59 Kansas City\'s most devastating floods Pause 2:07 KC Police say they\'re still looking for Kara Kopetsky, 17, and Jessica Runions, 21, who went missing in September 2016. The families of both women have been in the area. The Missouri State Highway Patrol said Tuesday afternoon. "'}]

In [28]:
df_test = dataset['test'].to_pandas()

In [29]:
df_test.columns = ['inputs', 'summary']

In [30]:
df_test.head()

,inputs,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...


In [40]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

322

In [36]:
with mlflow.start_run() as run:

    results = mlflow.evaluate(
         model_uri,
         df_test[:10],
        targets="summary",  # specify which column corresponds to the expected output
        model_type="text-summarization",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

2024/11/21 17:52:58 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2024/11/21 17:53:02 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2024/11/21 17:54:03 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run grandiose-bug-824 at: https://adb-2467347032368999.19.azuredatabricks.net/ml/experiments/837187481682972/runs/4f5429e4ab7e479f8007637b61ac9fbe
🧪 View experiment at: https://adb-2467347032368999.19.azuredatabricks.net/ml/experiments/837187481682972


{'toxicity/v1/mean': 0.001971986719581764,
 'toxicity/v1/variance': 2.6699768090500807e-06,
 'toxicity/v1/p90': 0.004177085403352976,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 8.24,
 'flesch_kincaid_grade_level/v1/variance': 4.626400000000001,
 'flesch_kincaid_grade_level/v1/p90': 10.94,
 'ari_grade_level/v1/mean': 10.78,
 'ari_grade_level/v1/variance': 7.945600000000001,
 'ari_grade_level/v1/p90': 13.6,
 'rouge1/v1/mean': 0.2911511575750777,
 'rouge1/v1/variance': 0.005443754674541205,
 'rouge1/v1/p90': 0.3537939297124601,
 'rouge2/v1/mean': 0.07808118158615665,
 'rouge2/v1/variance': 0.0024741511638419816,
 'rouge2/v1/p90': 0.1488422415013448,
 'rougeL/v1/mean': 0.15508871889419978,
 'rougeL/v1/variance': 0.002053453915922367,
 'rougeL/v1/p90': 0.21050319488817892,
 'rougeLsum/v1/mean': 0.15508871889419978,
 'rougeLsum/v1/variance': 0.002053453915922367,
 'rougeLsum/v1/p90': 0.21050319488817892}

In [41]:
from mlflow.metrics import latency
from mlflow.metrics.genai import answer_correctness

In [45]:
mlflow.enable_system_metrics_logging()


In [47]:
with mlflow.start_run() as run:

    results = mlflow.evaluate(
         model_uri,
         df_test[:10],
        targets="summary",  # specify which column corresponds to the expected output
        model_type="text-summarization",  # model type indicates which metrics are relevant for this task
        evaluators="default",
         extra_metrics=[

        latency(),
    ],
    )
results.metrics

2024/11/21 19:47:55 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


2024/11/21 19:48:17 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2024/11/21 19:48:21 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2024/11/21 19:49:22 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run righteous-shad-441 at: https://adb-2467347032368999.19.azuredatabricks.net/ml/experiments/837187481682972/runs/ea5cf79203754a459a9cbdbf6e251d95
🧪 View experiment at: https://adb-2467347032368999.19.azuredatabricks.net/ml/experiments/837187481682972


2024/11/21 19:49:29 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 19:49:30 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


{'latency/mean': 6.179589557647705,
 'latency/variance': 0.10555955221127304,
 'latency/p90': 6.747324275970459,
 'toxicity/v1/mean': 0.001971986719581764,
 'toxicity/v1/variance': 2.6699768090500807e-06,
 'toxicity/v1/p90': 0.004177085403352976,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 8.24,
 'flesch_kincaid_grade_level/v1/variance': 4.626400000000001,
 'flesch_kincaid_grade_level/v1/p90': 10.94,
 'ari_grade_level/v1/mean': 10.78,
 'ari_grade_level/v1/variance': 7.945600000000001,
 'ari_grade_level/v1/p90': 13.6,
 'rouge1/v1/mean': 0.2911511575750777,
 'rouge1/v1/variance': 0.005443754674541205,
 'rouge1/v1/p90': 0.3537939297124601,
 'rouge2/v1/mean': 0.07808118158615665,
 'rouge2/v1/variance': 0.0024741511638419816,
 'rouge2/v1/p90': 0.1488422415013448,
 'rougeL/v1/mean': 0.15508871889419978,
 'rougeL/v1/variance': 0.002053453915922367,
 'rougeL/v1/p90': 0.21050319488817892,
 'rougeLsum/v1/mean': 0.15508871889419978,
 'rougeLsum/v1/variance': 0.00205345391592

In [48]:
results.metrics

{'latency/mean': 6.179589557647705,
 'latency/variance': 0.10555955221127304,
 'latency/p90': 6.747324275970459,
 'toxicity/v1/mean': 0.001971986719581764,
 'toxicity/v1/variance': 2.6699768090500807e-06,
 'toxicity/v1/p90': 0.004177085403352976,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 8.24,
 'flesch_kincaid_grade_level/v1/variance': 4.626400000000001,
 'flesch_kincaid_grade_level/v1/p90': 10.94,
 'ari_grade_level/v1/mean': 10.78,
 'ari_grade_level/v1/variance': 7.945600000000001,
 'ari_grade_level/v1/p90': 13.6,
 'rouge1/v1/mean': 0.2911511575750777,
 'rouge1/v1/variance': 0.005443754674541205,
 'rouge1/v1/p90': 0.3537939297124601,
 'rouge2/v1/mean': 0.07808118158615665,
 'rouge2/v1/variance': 0.0024741511638419816,
 'rouge2/v1/p90': 0.1488422415013448,
 'rougeL/v1/mean': 0.15508871889419978,
 'rougeL/v1/variance': 0.002053453915922367,
 'rougeL/v1/p90': 0.21050319488817892,
 'rougeLsum/v1/mean': 0.15508871889419978,
 'rougeLsum/v1/variance': 0.00205345391592